In [3]:
import random
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [4]:
# Загрузка датасета
dataset = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
'''
Задаем функцию для оценки качества модели по метрикам:
accuracy, f1, precision, recall
'''
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [8]:
'''
Оцениваем качество модели до дообучения
'''
trainer = Trainer(
   model=model,
   args=TrainingArguments(
       "test_trainer",
       eval_strategy="no",  # Избегаем предупреждения об устаревшем evaluation_strategy
       report_to="none"  # Отключаем wandb
   ),
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["test"],
   compute_metrics=compute_metrics,
)

initial_metrics = trainer.evaluate()
print("Начальные метрики:", initial_metrics)

Начальные метрики: {'eval_loss': 1.4314956665039062, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.25276315789473686, 'eval_f1': 0.1082691161324459, 'eval_precision': 0.28777855520026624, 'eval_recall': 0.25276315789473686, 'eval_runtime': 56.4955, 'eval_samples_per_second': 134.524, 'eval_steps_per_second': 16.816}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
'''
Настраиваем параметры и запускаем дообучение с ускоренными параметрами
'''
training_args = TrainingArguments(
   output_dir="./results",
   num_train_epochs=1,  # Уменьшаем количество эпох
   per_device_train_batch_size=16,  # Увеличиваем размер батча
   per_device_eval_batch_size=16,
   eval_strategy="no",
   save_strategy="no",
   report_to="none",
   logging_steps=100,  # Реже логируем
   gradient_accumulation_steps=2  # Ускоряем обучение
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["test"],
   compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss
100,1.014700
200,0.649500
300,0.568900
400,0.526100
500,0.519800
600,0.465800
700,0.451100
800,0.464000
900,0.427400
1000,0.396900


TrainOutput(global_step=3750, training_loss=0.4220069783528646, metrics={'train_runtime': 2373.2687, 'train_samples_per_second': 50.563, 'train_steps_per_second': 1.58, 'total_flos': 3.157389361152e+16, 'train_loss': 0.4220069783528646, 'epoch': 1.0})

In [10]:
'''
Оцениваем финальное качество модели
'''
final_metrics = trainer.evaluate()
print("Финальные метрики:", final_metrics)
print("\nУлучшение:")
for metric in initial_metrics:
    if metric in final_metrics:
        diff = final_metrics[metric] - initial_metrics[metric]
        print(f"{metric}: {diff:+.4f}")

Финальные метрики: {'eval_loss': 0.16381555795669556, 'eval_accuracy': 0.9453947368421053, 'eval_f1': 0.9454344191943928, 'eval_precision': 0.9455870141344063, 'eval_recall': 0.9453947368421053, 'eval_runtime': 53.1951, 'eval_samples_per_second': 142.87, 'eval_steps_per_second': 8.929, 'epoch': 1.0}

Улучшение:
eval_loss: -1.2677
eval_accuracy: +0.6926
eval_f1: +0.8372
eval_precision: +0.6578
eval_recall: +0.6926
eval_runtime: -3.3004
eval_samples_per_second: +8.3460
eval_steps_per_second: -7.8870


Анализ результатов обучения модели:

- Точность (accuracy) улучшилась на +0.69%, достигнув ~94.5%
- Loss уменьшился на -1.27, что указывает на успешное обучение
- F1-score вырос на +0.84, показывая улучшение баланса между precision и recall
- Precision и recall увеличились на +0.66% и +0.69% соответственно

Вывод: Модель успешно дообучилась даже на 1 эпохе, показав улучшение по всем ключевым метрикам. Особенно важен рост F1-score, указывающий на сбалансированное улучшение качества классификации.